In [1]:
import numpy as np
import cmath

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.image as mpimg
from PIL import Image

from tqdm import tqdm

import os

from scipy.signal import find_peaks
import pandas as pd

import Functions.PlotFunctions as plFKT

import plotly.express as px
import plotly.graph_objects as go

In [2]:
measD="Data/"
measP="Images/"

In [5]:
messID=0

records=os.listdir(measD+'uff/')
take=messID

mess=records[take][:-4];
mess2=records[1][:-4];


print(mess)

Diatomischer Kristall Kurs SoSe 23


#### mittlere IRF berechnen + peaks

In [7]:
Fs=0
nPoints=np.zeros(len(records),int)
fmax=5000

Peaks=[]

MeanSpecS=np.zeros((len(records),fmax))
MeanSpecS_2=np.zeros((len(records),fmax))

# Daten laden
X = np.load(measD + 'NumPyArrays/'+ mess + '_Xnodes.npy')
IRFs = np.load(measD + 'NumPyArrays/'  + mess + '_IRFs.npy')

X_2 = np.load(measD + 'NumPyArrays/'+ mess2 + '_Xnodes.npy')
IRFs_2 = np.load(measD + 'NumPyArrays/'  + mess2 + '_IRFs.npy')

#Fs=len(IRFs[1]) #Messzeit beträgt 1sec. Ansonsten gibt es Fehler!!!
nPoints[take]=len(X)
nPoints[1] = len(X_2)

MeanSpec=np.zeros(len(IRFs[1]),dtype=np.complex_)
MeanSpec_2=np.zeros(len(IRFs_2[1]),dtype=np.complex_)

for i in range(nPoints[take]):
        MeanSpec += np.abs(IRFs[i])

for i in range(nPoints[1]):
        MeanSpec_2 += np.abs(IRFs_2[i])

#mittleres Spektrum
MeanSpec/=nPoints[take]
MeanSpec_2/=nPoints[take]

spec=np.abs(MeanSpec)[0:fmax]
spec_2=np.abs(MeanSpec_2)[0:fmax]

spec=np.log(np.divide(spec,max(spec)))
spec_2=np.log(np.divide(spec_2,max(spec_2)))


MeanSpecS[take,:]=spec
MeanSpecS_2[take,:]=spec_2

In [10]:
thresh = -9   # min. Höhe der peaks in dB_FS
minProm= 0.1 # min Prominence eines peaks (Abstand zwischen peak maximum und dem nächsten minimum), stdmäßig so 1.7
Peaks=[]
peaks,_= find_peaks(spec, distance=1, prominence=minProm, height=thresh)
Peaks=np.append(Peaks,peaks)
Peaks2=np.int32(np.unique(Peaks))

#plt.plot(spec)
#plt.scatter(Peaks2,spec[Peaks2], c='r')


fig = go.Figure()
trace = go.Scatter(x=np.arange(fmax+1), y=spec, name="Diatomischer Kristall (PLA)")
trace_2 = go.Scatter(x=np.arange(fmax+1), y=spec_2, name="Diatomischer Kristall (PLA) - Messung 2")

fig.add_trace(trace)
fig.add_trace(trace_2)

fig.update_layout(
    title="Diatomischer Kristall (PLA)",
    xaxis_title="Frequency (Hz)",
    yaxis_title="Mobility (dBFS)",
    #legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="RebeccaPurple"
    )
)

fig.show()

fig.write_html("Diatomischer_Kristall.html")

In [15]:
# Offsets Erklärung: Je X und Y für eine der 6 Messungen

OffSets=[(-110,-40),(45,90),(10,40),(0,40),(-45,0),(-30,10)]
Scale=[(.6,.6),(.45,.45),(.5,.5),(.5,.5),(.55,.55),(.55,.55)]

fmin=200 
Peaks2=Peaks2[Peaks2>=fmin]

PEAK=Peaks2

#### Daten Plotten

In [22]:
#layering FKT

def Layer(f):

    #imgData = Image.open(measP+'temp/'  + name + '_Re_' + str(f).zfill(4) + 'Hz.png')

    x2, y2 = imgData.size
    x2=int(round(x2*Scale[take][0]))
    y2=int(round(y2*Scale[take][1]))
    imgData=imgData.resize((x2,y2))

    imgData=plFKT.transparencyMode(imgData)
    offSet=OffSets[take]

    i3 = Image.new("RGBA",(x2,y2), (0,0,0,0))
    #i3.paste(bg, (0,0), bg)
    i3.paste(imgData, offSet, imgData)
    #i3.paste(mask, (0,0), mask)

    i3.save(measP+name+'/'+name+ str(f).zfill(4)+"Hz_mix.png", format="png")
    
mess=records[take][:-4]
name=mess

# Daten laden
X = np.load(measD + 'NumPyArrays/' + mess + '_Xnodes.npy')
Y = np.load(measD + 'NumPyArrays/'+ mess + '_Ynodes.npy')
# IRFs = np.load(measD + 'NumPyArrays/'+subFolder  + mess + '_IRFs.npy')

# Xn=[i for j, i in enumerate(X) if j not in Ignore]
# Yn=[i for j, i in enumerate(Y) if j not in Ignore]

Xn=X
Yn=Y

# measT="../Plots/"

# bg = Image.open(measP+"img/"+mess+"_cut.png").convert("RGBA")
#bg = Image.open(measP+mess+"/"+mess+".png").convert("RGBA")
#mask = Image.open(measP+mess+"/"+mess+"_weiß.png")

x1 = 1000
y1 = 1000

f=PEAK[6]

Amp=IRFs[:,f]
plotData=plFKT.PlotModeRe(Xn, Yn,Amp, (x1/ 50,y1/50))
plotData.savefig(measP+'temp/'  + name +'_Re_' + str(f).zfill(4) + 'Hz.png')
#Layer(f)

FileNotFoundError: [Errno 2] No such file or directory: 'Images/Gitarre_mit_kleineren_Magneten_2x5mm_Messung_2/Gitarre_mit_kleineren_Magneten_2x5mm_Messung_2.png'

In [17]:
# save all modes

for f in PEAK:
    Amp=IRFs[:,f]
    ph=[cmath.phase(i) for i in Amp]
    ab=np.abs(Amp)

    bins=np.zeros(360)

    for i in range(len(ph)):
        id=round(ph[i]/np.pi*180)+180
        if id==360:
            id=0
        bins[int(id)]=bins[int(id)]+1#+ab[i]

    ang0=(np.where(bins==max(bins))[0][0]-180)/180*np.pi
    ang0
    ph=ph-ang0
    for (i,p) in enumerate(ph):
        if p<-np.pi:
            p=p+2*np.pi
        elif p>np.pi:
            p=p-2*np.pi
        
        if p>np.pi/2:
            p=p-np.pi
        elif p<=-np.pi/2:
            p=p+np.pi
        ph[i]=p
    dph=np.mean(ph)+ang0

    for (i,a) in enumerate(Amp):
        Amp[i]=a*cmath.exp(dph*1j*(-1))

    plotData=plFKT.PlotModeAbs(Xn, Yn,Amp, (12,2))
    plotData.savefig(measP+'temp/'   + name + '_Abs_' + str(f).zfill(4) + 'Hz.png')

    plotData=plFKT.PlotModeRe(Xn, Yn,Amp, (x1/50,y1/50))
    plotData.savefig(measP+'temp/'   + name + '_Re_' + str(f).zfill(4) + 'Hz.png')
    Layer(f)

NameError: name 'x1' is not defined

In [ ]:
mydir=measP+"temp/"
filelist = [ f for f in os.listdir(mydir)]
for f in filelist:
    os.remove(os.path.join(mydir, f))